# Test Redshift Database

## Imports

In [1]:
import os
import glob
from pytz import utc
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from desiutil.iers import freeze_iers
from desiutil.log import get_logger, DEBUG, INFO
from desispec.io.meta import faflavor2program
from desispec.io.util import checkgzip
import desispec.database.redshift as dsr
freeze_iers()

INFO:iers.py:82:freeze_iers: Freezing IERS table used by astropy time, coordinates.


## Important Setup

In [2]:
specprod = os.environ['SPECPROD'] = 'fuji'
overwrite = False

## Consistency checks on exposures, frames and tiles

### Load two versions of tiles file

In [3]:
tiles_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.fits'), hdu='TILE_COMPLETENESS')

In [4]:
tiles_fits

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int32,bytes20,bytes6,bytes20,bytes20,int64,float64,float64,float64,float64,float64,float64,float64,bytes20,float64,float64,float64,float64,bytes20,float64,int32
80615,cmx,other,m33,cmxm33,4,3600.1,24.027,31.39,0.0,3390.9,0.0,1000.0,obsend,3361.7,3390.9,3320.2,3878.6,other,0.9,20201216
80609,sv1,dark,lrgqso,cmxlrgqso,15,13500.7,150.12,2.206,0.0,7828.3,0.0,4000.0,obsstart,7847.0,7828.3,8057.3,10707.3,dark,0.9,20201217
80607,sv1,dark,lrgqso,cmxlrgqso,15,13500.8,106.74,56.1,0.0,9374.2,0.0,4000.0,obsstart,8794.6,9374.2,8553.9,8541.1,dark,0.9,20201219
80606,sv1,dark,elg,cmxelg,12,10800.7,36.448,-4.501,0.0,6699.4,0.0,4000.0,obsstart,6555.6,6699.4,6583.6,7397.5,dark,0.9,20201219
80620,sv1,dark,lrgqso,sv1lrgqso,9,8100.5,144.0,65.0,0.0,7128.7,0.0,4000.0,obsstart,6643.9,7128.7,6299.8,6316.1,dark,0.9,20201221
80622,sv1,dark,lrgqso,sv1lrgqso,6,5400.3,155.0,32.325,0.0,4916.9,0.0,4000.0,obsstart,4772.1,4916.9,4739.8,6935.4,dark,0.9,20201221
80623,sv1,dark,elg,sv1elg,8,7200.4,155.0,32.425,0.0,4969.7,0.0,4000.0,obsstart,4903.1,4969.7,4916.9,7266.2,dark,0.9,20201221
80611,sv1,bright,bgsmws,sv1bgsmws,12,3600.9,35.22,30.3,0.0,669.2,0.0,600.0,obsstart,641.1,723.7,669.2,407.7,bright,0.9,20201222
80617,sv1,bright,bgsmws,sv1bgsmws,16,4801.0,5.0,28.0,0.0,1309.5,0.0,600.0,obsstart,1251.7,1369.7,1309.5,884.1,bright,0.9,20201223


In [5]:
tiles_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [6]:
tiles_csv

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str6,float64,int64
80615,cmx,other,m33,cmxm33,4,3600.1,24.027,31.39,0.0,3390.9,0.0,1000.0,obsend,3361.7,3390.9,3320.2,3878.6,other,0.9,20201216
80609,sv1,dark,lrgqso,cmxlrgqso,15,13500.7,150.12,2.206,0.0,7828.3,0.0,4000.0,obsstart,7847.0,7828.3,8057.3,10707.3,dark,0.9,20201217
80607,sv1,dark,lrgqso,cmxlrgqso,15,13500.8,106.74,56.1,0.0,9374.2,0.0,4000.0,obsstart,8794.6,9374.2,8553.9,8541.1,dark,0.9,20201219
80606,sv1,dark,elg,cmxelg,12,10800.7,36.448,-4.501,0.0,6699.4,0.0,4000.0,obsstart,6555.6,6699.4,6583.6,7397.5,dark,0.9,20201219
80620,sv1,dark,lrgqso,sv1lrgqso,9,8100.5,144.0,65.0,0.0,7128.7,0.0,4000.0,obsstart,6643.9,7128.7,6299.8,6316.1,dark,0.9,20201221
80622,sv1,dark,lrgqso,sv1lrgqso,6,5400.3,155.0,32.325,0.0,4916.9,0.0,4000.0,obsstart,4772.1,4916.9,4739.8,6935.4,dark,0.9,20201221
80623,sv1,dark,elg,sv1elg,8,7200.4,155.0,32.425,0.0,4969.7,0.0,4000.0,obsstart,4903.1,4969.7,4916.9,7266.2,dark,0.9,20201221
80611,sv1,bright,bgsmws,sv1bgsmws,12,3600.9,35.22,30.3,0.0,669.2,0.0,600.0,obsstart,641.1,723.7,669.2,407.7,bright,0.9,20201222
80617,sv1,bright,bgsmws,sv1bgsmws,16,4801.0,5.0,28.0,0.0,1309.5,0.0,600.0,obsstart,1251.7,1369.7,1309.5,884.1,bright,0.9,20201223


### Are the two tiles files self-consistent?

In [7]:
for row in range(len(tiles_fits)):
    for col in tiles_fits.colnames:
        try:
            assert tiles_fits[row][col] == tiles_csv[row][col]
        except AssertionError:
            print(tiles_fits[row]['TILEID'], col, tiles_fits[row][col], tiles_csv[row][col])

### Load two versions of exposures file

In [8]:
exposures_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [9]:
exposures_fits

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes16,bytes19,float64,float64,float64,bytes6,float64,float32,float64,float64,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20201214,67710,80605,36.448,-4.601,59198.17546592,sv1,dark,lrgqso,cmxlrgqso,900.0768432617188,17.298908233642578,4000.0,dark,0.9,1.244535,0.027954712510108948,0.0,0.0,2.0115008,0.4892555,1.3297243,1.2010275246488162,125.88352,176.69063,35.75251,312.16226,16.392647,17.298908,17.623693,13.739464269950634,15.295618,18.778387,22.87816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.147212948116103,21.134354619896946,19.340615345357953,0.0,0.0,0.0,0.0
20201214,67711,80605,36.448,-4.601,59198.18926673,sv1,dark,lrgqso,cmxlrgqso,900.0641479492188,15.429797172546387,4000.0,dark,0.9,1.249668,0.027954712510108948,0.0,0.0,1.7941624,0.42504597,1.2019708,1.0799514889833517,115.25519,159.52464,32.855064,275.14603,14.817721,15.429797,16.135727,12.354383718645755,13.809607,17.256554,20.165266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.151793391382224,21.134922261901163,19.317079553988044,0.0,0.0,0.0,0.0
20201214,67712,80605,36.448,-4.601,59198.20253263,sv1,dark,lrgqso,cmxlrgqso,900.0560913085938,34.315513610839844,4000.0,dark,0.9,1.264157,0.027954712510108948,0.0,0.0,3.990176,0.9686168,2.6183681,2.379345850396203,246.09148,344.20648,69.03682,587.11316,32.278862,34.315514,34.45281,27.219140799402464,29.797003,36.260395,43.029125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.157862150878422,21.13464540199608,19.321768539212346,0.0,0.0,0.0,0.0
20201214,67713,80605,36.448,-4.601,59198.21581194,sv1,dark,lrgqso,cmxlrgqso,900.0040283203125,43.364559173583984,4000.0,dark,0.9,1.288657,0.027954712510108948,0.0,0.0,5.0423903,1.2342193,3.3316402,2.9712147275102323,310.99176,441.2709,87.99428,744.9247,41.071976,43.36456,43.538845,33.98997753936961,38.1996,46.21747,54.595024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.157039666465575,21.16149192467248,19.40011719034235,0.0,0.0,0.0,0.0
20201214,67733,80606,36.448,-4.501,59198.2450305,sv1,dark,elg,cmxelg,900.0259399414062,107.48906707763672,4000.0,dark,0.9,1.380783,0.027508262544870377,0.0,0.0,12.498728,3.059576,8.157001,7.830726769170724,757.43445,1093.0906,214.4819,1706.7959,100.55831,107.48907,106.040825,89.58161944225778,94.625824,112.6529,125.08991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.11347440490316,21.109118127481352,19.362904182378095,0.0,0.0,0.0,0.0
20201214,67744,80607,106.74,56.1,59198.26702079,sv1,dark,lrgqso,cmxlrgqso,900.0259399414062,300.8516540527344,4000.0,dark,0.9,1.22422,0.05611468106508255,0.0,0.0,34.98275,8.371301,22.693335,19.880483982246368,2024.0605,2953.2625,561.0279,3931.1772,279.76013,300.85165,283.36847,227.42792628609314,255.65575,294.67017,288.1133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.18968242996683,21.24604411535422,19.607193720412404,0.0,0.0,0.0,0.0
20201214,67765,80607,106.74,56.1,59198.33230605,sv1,dark,lrgqso,cmxlrgqso,900.0891723632812,289.2614440917969,4000.0,dark,0.9,1.111614,0.05611468106508255,0.0,0.0,33.63505,8.169466,21.862267,19.924564858619124,1950.2588,2926.148,553.4403,3952.5176,269.51483,289.26144,273.03622,227.93220084556978,253

In [10]:
exposures_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.csv'), format='ascii.csv')

In [11]:
exposures_csv

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int64,int64,int64,float64,float64,float64,str7,str6,str16,str19,float64,float64,float64,str6,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20201214,67710,80605,36.448,-4.601,59198.17546592,sv1,dark,lrgqso,cmxlrgqso,900.1,17.3,4000.0,dark,0.9,1.245,0.028,0.0,0.0,2.0,0.5,1.3,1.2,125.9,176.7,35.8,312.2,16.4,17.3,17.6,13.7,15.3,18.8,22.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.147,21.134,19.341,0.0,0.0,0.0,0.0
20201214,67711,80605,36.448,-4.601,59198.18926673,sv1,dark,lrgqso,cmxlrgqso,900.1,15.4,4000.0,dark,0.9,1.25,0.028,0.0,0.0,1.8,0.4,1.2,1.1,115.3,159.5,32.9,275.1,14.8,15.4,16.1,12.4,13.8,17.3,20.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.152,21.135,19.317,0.0,0.0,0.0,0.0
20201214,67712,80605,36.448,-4.601,59198.20253263,sv1,dark,lrgqso,cmxlrgqso,900.1,34.3,4000.0,dark,0.9,1.264,0.028,0.0,0.0,4.0,1.0,2.6,2.4,246.1,344.2,69.0,587.1,32.3,34.3,34.5,27.2,29.8,36.3,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.158,21.135,19.322,0.0,0.0,0.0,0.0
20201214,67713,80605,36.448,-4.601,59198.21581194,sv1,dark,lrgqso,cmxlrgqso,900.0,43.4,4000.0,dark,0.9,1.289,0.028,0.0,0.0,5.0,1.2,3.3,3.0,311.0,441.3,88.0,744.9,41.1,43.4,43.5,34.0,38.2,46.2,54.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.157,21.161,19.4,0.0,0.0,0.0,0.0
20201214,67733,80606,36.448,-4.501,59198.2450305,sv1,dark,elg,cmxelg,900.0,107.5,4000.0,dark,0.9,1.381,0.028,0.0,0.0,12.5,3.1,8.2,7.8,757.4,1093.1,214.5,1706.8,100.6,107.5,106.0,89.6,94.6,112.7,125.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.113,21.109,19.363,0.0,0.0,0.0,0.0
20201214,67744,80607,106.74,56.1,59198.26702079,sv1,dark,lrgqso,cmxlrgqso,900.0,300.9,4000.0,dark,0.9,1.224,0.056,0.0,0.0,35.0,8.4,22.7,19.9,2024.1,2953.3,561.0,3931.2,279.8,300.9,283.4,227.4,255.7,294.7,288.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.19,21.246,19.607,0.0,0.0,0.0,0.0
20201214,67765,80607,106.74,56.1,59198.33230605,sv1,dark,lrgqso,cmxlrgqso,900.1,289.3,4000.0,dark,0.9,1.112,0.056,0.0,0.0,33.6,8.2,21.9,19.9,1950.3,2926.1,553.4,3952.5,269.5,289.3,273.0,227.9,253.3,290.7,289.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.192,21.177,19.416,0.0,0.0,0.0,0.0
20201214,67766,80607,106.74,56.1,59198.34618685,sv1,dark,lrgqso,cmxlrgqso,900.1,284.4,4000.0,dark,0.9,1.102,0.056,0.0,0.0,33.1,8.0,21.2,19.7,1887.7,2800.9,529.9,3843.4,261.8,284.4,264.3,225.2,242.5,278.3,281.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.184,21.135,19.313,0.0,0.0,0.0,0.0
20201214,67767,80607,106.74,56.1,59198.35939494,sv1,dark,lrgqso,cmxlrgqso,900.1,308.7,4000.0,dark,0.9,1.097,0.056,0.0,0.0,35.9,8.7,23.4,20.4,2070.5,3104.4,586.7,4141.9,287.9,308.7,289.9,233.4,268.7,308.1,303.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.19,21.168,19.355,0.0,0.0,0.0,0.0


### Are the two exposures files self-consistent?

In [12]:
for row in range(len(exposures_fits)):
    for col in exposures_fits.colnames:
        try:
            assert exposures_fits[row][col] == exposures_csv[row][col]
        except AssertionError:
            try:
                assert np.around(exposures_fits[row][col].astype(float), 1) == exposures_csv[row][col]
            except AssertionError:
                try:
                    assert np.around(exposures_fits[row][col].astype(float), 2) == exposures_csv[row][col]
                except AssertionError:
                    try:
                        assert np.around(exposures_fits[row][col].astype(float), 3) == exposures_csv[row][col]
                    except AssertionError:
                        print(exposures_fits[row]['TILEID'], col, exposures_fits[row][col], exposures_csv[row][col])

### What programs are present?

In [13]:
np.unique(faflavor2program(exposures_fits['FAFLAVOR']))

array(['backup', 'bright', 'dark', 'other'], dtype='<U6')

In [14]:
np.unique(exposures_fits['PROGRAM'])

backup
bright
dark
other


In [15]:
np.unique(exposures_fits['GOALTYPE'])

backup
bright
dark
other


In [16]:
np.unique(faflavor2program(tiles_fits['FAFLAVOR']))

array(['backup', 'bright', 'dark', 'other'], dtype='<U6')

In [17]:
np.unique(tiles_fits['PROGRAM'])

backup
bright
dark
other


In [18]:
assert (faflavor2program(tiles_fits['FAFLAVOR']) == tiles_fits['PROGRAM']).all

In [19]:
program = faflavor2program(exposures_fits['FAFLAVOR'])
assert (exposures_fits['PROGRAM'] == program).all()

In [20]:
for survey in np.unique(exposures_fits['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures_fits['SURVEY'] == survey]).tolist(), ',', sep='')

'cmx': ['other'],
'special': ['dark'],
'sv1': ['backup', 'bright', 'dark', 'other'],
'sv2': ['backup', 'bright', 'dark'],
'sv3': ['backup', 'bright', 'dark'],


### Compare frames to exposures

In [21]:
for expid in frames['EXPID']:
    assert expid in exposures_fits['EXPID']

In [22]:
for k, expid in enumerate(exposures_fits['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures_fits[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures_fits[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures_fits[k]['TILEDEC']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures_fits[k]['AIRMASS']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['SEEING_ETC']).all()
    try:
        assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['EFFTIME_ETC']).all()
    except AssertionError:
        print('EFFTIME_ETC', expid, exposures_fits[k]['TILEID'], exposures_fits[k]['EFFTIME_ETC'], frames[column][frames['EFFTIME_ETC'] == expid].tolist())
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures_fits[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures_fits[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures_fits[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures_fits[k]['MINTFRAC']).all()
    for column in ('MJD', 'EXPTIME', 'GOALTIME'):
        if column == 'GOALTIME' and (frames[column][frames['EXPID'] == expid] == 0).all():
            print(f"GOALTIME discrepancy for {expid}.")
        else:
            try:
                assert (np.around(frames[column][frames['EXPID'] == expid], 2) == np.around(exposures_fits[k][column], 2)).all()
            except AssertionError:
                try:
                    assert (np.around(frames[column][frames['EXPID'] == expid], 3) == np.around(exposures_fits[k][column], 3)).all()
                except AssertionError:
                    try:
                        assert (np.around(frames[column][frames['EXPID'] == expid], 4) == np.around(exposures_fits[k][column], 4)).all()
                    except AssertionError:
                        pass
                        # print(column, expid, exposures_fits[k][column], frames[column][frames['EXPID'] == expid].tolist())

### Compare tiles to exposures

In [23]:
for row in tiles_fits:
    w = exposures_fits['TILEID'] == row['TILEID']
    assert len(exposures_fits[w]) == row['NEXP']
    for column in ('SURVEY', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        try:
            assert (exposures_fits[w][column] == row[column]).all()
        except AssertionError:
            print(row['TILEID'], row[column])
            print(exposures_fits[w][column])
    for column in ('TILERA', 'TILEDEC'):
        try:
            assert (np.around(exposures_fits[w][column], 2) == row[column]).all()
        except AssertionError:
            try:
                assert (np.around(exposures_fits[w][column], 3) == row[column]).all()
            except AssertionError:
                assert (np.around(exposures_fits[w][column], 4) == row[column]).all()
    for column in ('EXPTIME', 'EFFTIME_SPEC', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
        except AssertionError:
            print(row['TILEID'], row[column])
            print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('EFFTIME_ETC', 'EFFTIME_GFA'):
        if (exposures_fits[w][column] == 0).any():
            assert row[column] == 0
        else:
            try:
                assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('GOALTIME', 'MINTFRAC'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column], 1), row[column])
        except AssertionError:
            try:
                assert np.allclose(np.around(exposures_fits[w][column], 2), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.round(exposures_fits[w][column], decimals=1))
    
    assert exposures_fits[w]['NIGHT'].max() == row['LASTNIGHT']

80873 2643.1
LRG_EFFTIME_DARK 2643.2
248 1230.7
BGS_EFFTIME_BRIGHT 1230.6
80862 7748.5
BGS_EFFTIME_BRIGHT 7748.4
